In [234]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import xml.etree.ElementTree as ET
import requests
import time


cell_journals = ['current-biology','cell-systems','cell-metabolism','neuron']
trend_journals = ['endocrinology-metabolism']
cellurllist = ['http://www.cell.com/' + i + '/inpress.rss' for i in cell_journals]
trendurllist = ['https://www.cell.com/trends/' + j + '/inpress.rss' for j in trend_journals]
urllist = cellurllist + trendurllist
keywords = ['anxiety','estrogen','corticosterone','depression','brain']


def find_relevant_articles(url, keywords):
    conn = urlopen(url)
    data = conn.read()
    conn.close()
    less_heinous_rsscheck = BeautifulSoup(data,"xml")
    total_article_list = less_heinous_rsscheck.find_all("item")
    getit = lambda f,l: sum([int(j.lower() in str(f.find('description')).lower()) for j in l])
    relevant_articles = [i for i in total_article_list if getit(i,keywords) >= 2] # needing multiple hits should reduce false positives in case of broader search terms
    return [total_article_list,relevant_articles]

def get_those_sources(itemlist,keywords): 
    sourcesdict = {}
    for article in itemlist: 
        workingtitle = str(article.find_all("dc:title"))[11:-12]
        sourcesdict[workingtitle] = []
        currenturl = str(article.find('link'))[6:-7]
        conn = requests.get(currenturl) #using requests.get to avoid a 403 Forbidden error
        data = conn.text
        time.sleep(2) # slows down request rate to resemble normal-ish traffic. Hopefully. 
        htmlcheck = BeautifulSoup(data,"xml")
        citations = [content for content in htmlcheck.find_all("meta") if 'citation_title=' in str(content)]
        #print(citations[0])
        for cite in citations: 
            citelist = []
            rawlist = str(cite).split('=')
            title = rawlist[2].split(';')[0]
            #foundit = any([True if j in title else False for j in keywords])
            keyfound = [1 if j in title else 0 for j in keywords]
            foundit = sum(keyfound) >= 1 # if we're searching through a relevant paper we shouldn't need to trim down the relevant citations
            if foundit: 
                author = rawlist[3].split(';')[0]
                journal = rawlist[5].split(';')[0]
                year = rawlist[-2].split(';')[0]
                citelist = [title,journal,author,year]
                sourcesdict[workingtitle].append(citelist)    
    return sourcesdict

def just_the_titles(itemlist): 
    outlist = ['I found the following titles: \n\n ----------\n']
    index = 1
    for i in itemlist: 
        outlist.append(str(index) + '. ' + str(i.find('title'))[7:-8])
        index += 1
    return outlist

def prettify_titles_and_citations(citation_dict): 
    outstr = 'I found the following titles: \n --------- \n'
    index = 1
    allcitations = 0
    for i in citation_dict.keys():
        outstr += str(index) + '. ' + i + '. It cited: \n\n'
        index += 1
        for citation in citation_dict[i]: 
            #print(citation)
            outstr += '\t Title: ' + citation[0] + '\n'
            outstr += '\t Journal: ' + citation[1] + '\n'
            outstr += '\t Primary Author: ' + citation[2] + '\n'
            outstr += '\t Year Published: ' + citation[3] + '\n\n'
            allcitations += 1
    outstr += '\n All in all there were ' + str(len(citation_dict.keys())) + ' parent articles and ' + str(allcitations) + ' citations found.'
    return(outstr)
        

all_articles = []
relevant_articles = []

for url in urllist: 
    #print(url)
    [t,r] = find_relevant_articles(url,keywords)
    all_articles.extend(t)
    relevant_articles.extend(r)

startoff_list = get_those_sources(relevant_articles,keywords)

nice_citations = prettify_titles_and_citations(startoff_list)

print(nice_citations)

I found the following titles: 
 --------- 
1. Tibolone as Hormonal Therapy and Neuroprotective Agent. It cited: 

	 Title: Steroid hormones and their action in women’s brains: the importance of hormonal balance
	 Journal: Front. Public Health
	 Primary Author: J.P. Del Río
	 Year Published: 2018

	 Title: The neuroprotective actions of oestradiol and oestrogen receptors
	 Journal: Nat. Rev. Neurosci.
	 Primary Author: M.-A. Arevalo
	 Year Published: 2015

	 Title: Agonist activity of the 3-hydroxy metabolites of tibolone through the oestrogen receptor in the mouse N20.1 oligodendrocyte cell line and normal human astrocytes
	 Journal: J. Neuroendocrinol.
	 Primary Author: C.B. Guzmán
	 Year Published: 2018

	 Title: Metabolism of exogenous sex steroids and effect on brain functions with a focus on tibolone
	 Journal: J. Steroid Biochem. Mol. Biol.
	 Primary Author: H.A.M.M. Verheul
	 Year Published: 2006

	 Title: In vivo and in vitro estrogenic and progestagenic actions of tibolone
	 J